In [ ]:
using Rays

## Creating a scene

In [ ]:
# Define the scene
scene = Rays.Scene()

# Define a camera
camera = Rays.Camera()
camera.screen_res .= [500, 500]
push!(scene, camera)

# Define a cube at the origin
cube = Rays.Cube(zeros(Float32, 3), 1.0f0)
push!(scene, cube)

## Creating a parameters object

In [ ]:
julia_green = Float32[0.22, 0.596, 0.149]
julia_purple = Float32[0.584, 0.345, 0.698]
julia_red = Float32[0.796, 0.235, 0.2]
julia_colors = hcat(julia_green, julia_purple, julia_red)
color = Rays.get_canvas(camera; color = true)

mutable struct Params{F} <: Rays.Parameters{F}
    θ::F
    ϕ::F
    dθ::F
    dist::F
    colors::Matrix{F}
    color::Array{F,3}
end

params = Params(Float32(π/4), Float32(3π/8), 0.0f0, 7.5f0, julia_colors, color)

## Creating parameter update functions

In [ ]:
import SimpleDirectMediaLayer as SDL
using SimpleDirectMediaLayer.LibSDL2

"""
Actions:
- Change the horizontal rotation speed dθ around the cube (left/right keys)
- Change the vertical position of the camera dϕ (up/down)
"""
function affect_parameters_input!(params::Params{F}, event::SDL_Event, Δt::F)::Bool where {F}
    parameters_changed = false
    event_type = event.type

    if event_type == SDL.SDL_KEYDOWN
        scan_code = event.key.keysym.scancode 
        if scan_code == SDL.SDL_SCANCODE_RIGHT
            params.dθ += 2*Δt
            parameters_changed = true
        elseif scan_code == SDL.SDL_SCANCODE_LEFT
            params.dθ -= 2*Δt
            parameters_changed = true
        elseif scan_code == SDL.SDL_SCANCODE_UP
            params.ϕ = max(params.ϕ - Δt, 1e-3)
            parameters_changed = true
        elseif scan_code == SDL.SDL_SCANCODE_DOWN
            params.ϕ = min(params.ϕ + Δt, π-1e-3)
            parameters_changed = true
        end
    end
    return parameters_changed
end

function affect_parameters_time!(params::Params{F}, Δt::F)::Bool where {F}
    params.θ += params.dθ * Δt
    return true
end

## Creating a rendering function

In [ ]:
function get_render(scene::Rays.Scene{F}, params::Rays.Parameters{F})::Array{F, 3} where {F <: AbstractFloat}

    # Adjust camera
    Rays.look_at!(scene.cameras[1], zeros(Float32,3), params.dist, params.θ, params.ϕ)

    # Get intersection data
    intersection_data = Rays.shape_view(scene; data_variables = [:dim])

    # Get grayscale image
    canvas_grayscale = Rays.cam_is_source(intersection_data.t, dropoff_curve = x -> clamp(3.75-x/2, 0.0, 1.0));

    # Apply color to the image
    color .= 0.0
    Rays.add_color!(color, julia_colors, intersection_data.dim)
    canvas_color = Rays.apply_color(canvas_grayscale, color);

    return canvas_color
end 

## Creating an interactor

In [ ]:
interactor = Rays.Interactor(scene, params, affect_parameters_input!, affect_parameters_time!, get_render)

In [ ]:
Rays.run!(interactor)